In [19]:
%pylab --no-import-all
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from keras.utils.data_utils import get_file
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

import common as localUtils
from importlib import reload
reload(localUtils)

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


<module 'common' from '/home/arthur/kaggle/mine/DSB3Tutorial/model/common.py'>

In [20]:
images_path = '/data1/cancer/KAG2017/stage1_scaled/128'
tSet = localUtils.TrainSet(images_path,is3D=False)

In [21]:
class Vg3():
    """The Vg3 16 Imagenet model"""


    def __init__(self,shape,comment="here we go"):
        self.create(shape)
        self.meta = {"shape":str(shape),comment:comment}


    def predict(self, imgs, details=False):
        all_preds = self.model.predict(imgs)
        idxs = np.argmax(all_preds, axis=1)
        return np.array(preds), idxs
    
    def model_preprocess(self, x):
        return x

    def ConvBlock(self, layers, filters):
        model = self.model
        for i in range(layers):
            model.add(ZeroPadding2D((1, 1)))
            model.add(Convolution2D(filters, 3, 3, activation='relu' ,dim_ordering='th'))
        model.add(MaxPooling2D((2, 2), strides=(2, 2)))


    def FCBlock(self):
        model = self.model
        model.add(Dense(4096, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))


    def create(self, shape):

        model = self.model = Sequential()
        model.add(Lambda(self.model_preprocess, input_shape=shape))
        self.ConvBlock(2, 8)
        self.ConvBlock(2, 8)
        self.ConvBlock(2, 16)
        #self.ConvBlock(2, 64)
        #self.ConvBlock(2, 64)
        #self.ConvBlock(2, 128)
        #self.ConvBlock(3, 256)
        #self.ConvBlock(3, 512)
        #self.ConvBlock(3, 512)

        model.add(Flatten())
        #self.FCBlock()
        self.FCBlock()
        model.add(Dense(1, init='normal', activation='sigmoid'))
        #model.add(Dense(1, activation='sigmoid'))

    def fit(self, batches, val_batches, nb_epoch=1):
        self.model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=nb_epoch,
                validation_data=val_batches, nb_val_samples=val_batches.nb_sample)


    def test(self, path, batch_size=8):
        test_batches = self.get_batches(path, shuffle=False, batch_size=batch_size, class_mode=None)
        return test_batches, self.model.predict_generator(test_batches, test_batches.nb_sample)


In [22]:
vg =  Vg3(tSet.trainX()[0].shape,"2d trying another 8 block")
vg.model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_4 (Lambda)                (None, 128, 128, 128) 0           lambda_input_4[0][0]             
____________________________________________________________________________________________________
zeropadding2d_11 (ZeroPadding2D) (None, 128, 130, 130) 0           lambda_4[0][0]                   
____________________________________________________________________________________________________
convolution2d_11 (Convolution2D) (None, 8, 128, 128)   9224        zeropadding2d_11[0][0]           
____________________________________________________________________________________________________
zeropadding2d_12 (ZeroPadding2D) (None, 8, 130, 130)   0           convolution2d_11[0][0]           
___________________________________________________________________________________________

In [ ]:
vg.model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

/home/arthur/anaconda2/envs/lung3/lib/python3.5/site-packages/keras/engine/topology.py:1804: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "sequential_4_model" was not an Input tensor, it was generated by layer lambda_4.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: lambda_input_4:0
  str(x.name))


In [ ]:
localUtils.evaluate(vg,tSet,"basic2d",nb_epoch=7)
plt.show()